In [1]:
library(dplyr)
library(Seurat)
library(spacexr)

scRNA=readRDS('/csb2/finishedProject/HuiLiJian_DN_2025/10x_HD/5-final/2-RCTD/scRNA_ref.rds')
hd=readRDS('/csb2/finishedProject/HuiLiJian_DN_2025/10x_HD/5-final/2-RCTD/18s18941-T2.016um/18s18941-T2.016um.rds')
coords = read.csv('/csb2/finishedProject/HuiLiJian_DN_2025/10x_HD/5-final/2-RCTD/18s18941-T2.016um/18s18941-T2.016um.coords.csv',row.names=1)
# scRNA <- RenameIdents(scRNA, "NK/T" = "NKT")
    
RCTD_result <- create.RCTD(SpatialRNA(coords,
                                      GetAssayData(hd,assay='RNA',slot='counts'),use_fake_coords=FALSE), 
        Reference(GetAssayData(scRNA,assay='RNA',slot='counts'),Idents(scRNA)), 
        max_cores=20,test_mode=FALSE,MAX_MULTI_TYPES = 2,UMI_min = 0)
RCTD_result@config$max_cores <- 8
RCTD_result <- run.RCTD(RCTD_result,doublet_mode='doublet')
saveRDS(RCTD_result, file = '/csb2/finishedProject/HuiLiJian_DN_2025/10x_HD/5-final/2-RCTD/18s18941-T2.016um/RCTD_result.doublet.rds')
# write.csv(RCTD_result@results$weights,'/picb/lihonglab/tangzhixuan/HuiLiJian_DN_2025/RCTD/RCTD_result.csv')

# write.csv(normalize_weights(RCTD_result@results$weights),
#           paste0('/picb/lihonglab/tangzhixuan/HuiLiJian_DN_2025/RCTD/RCTD_normalized_result.csv'))
    


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Attaching SeuratObject

Warning message in Reference(GetAssayData(scRNA, assay = "RNA", slot = "counts"), :
“Reference: number of cells per cell type is 53010, larger than maximum allowable of 10000. Downsampling number of cells to: 10000”
Begin: process_cell_type_info

process_cell_type_info: number of cells in reference: 76120

process_cell_type_info: number of genes in reference: 36601




   Neutrophil           CAF         Tumor          CD4T    Macrophage 
        10000          6789         10000         10000         10000 
  Endothelial          CD8T Smooth muscle        B cell 
        10000         10000          2741          6590 


Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 2.7 GiB”
Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 1.9 GiB”
Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 2.7 GiB”
Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 2.7 GiB”
Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 2.7 GiB”
Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 2.7 GiB”
Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 2.7 GiB”
Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 1.8 GiB”
End: process_cell_type_info

create.RCTD: getting regression differentially expressed genes: 

get_de_genes: Neutrophil found DE genes: 538

get_de_genes: CAF found DE genes: 207

get_de_genes: Tumor found DE genes: 302

get

[1] "gather_results: finished 1000"
[1] "gather_results: finished 2000"
[1] "gather_results: finished 3000"
[1] "gather_results: finished 4000"
[1] "gather_results: finished 5000"
[1] "gather_results: finished 6000"
[1] "gather_results: finished 7000"
[1] "gather_results: finished 8000"
[1] "gather_results: finished 9000"
[1] "gather_results: finished 10000"
[1] "gather_results: finished 11000"
[1] "gather_results: finished 12000"
[1] "gather_results: finished 13000"
[1] "gather_results: finished 14000"
[1] "gather_results: finished 15000"
[1] "gather_results: finished 16000"
[1] "gather_results: finished 17000"
[1] "gather_results: finished 18000"
[1] "gather_results: finished 19000"
[1] "gather_results: finished 20000"
[1] "gather_results: finished 21000"
[1] "gather_results: finished 22000"
[1] "gather_results: finished 23000"
[1] "gather_results: finished 24000"
[1] "gather_results: finished 25000"
[1] "gather_results: finished 26000"
[1] "gather_results: finished 27000"
[1] "gathe

In [2]:
library(dplyr)
library(Seurat)
library(spacexr)
rctd_14u = readRDS('/csb2/finishedProject/HuiLiJian_DN_2025/10x_HD/5-final/2-RCTD/18s18941-T2.016um/RCTD_result.doublet.rds')
# 2. check the result of doublet mode
rctd_14u_res <- rctd_14u@results

head(rctd_14u_res$results_df)      # 'spot_class' may include 'reject', 'singlet', 'doublet_certain', 'doublet_uncertain'
head(rctd_14u_res$weights_doublet) # RCTD values of doublet mode

## 3. build RCTD matrix of doublet mode
rctd_14u_res_db <- rctd_14u_res$weights_doublet
dim(rctd_14u_res_db)

celltype_firsec <- rctd_14u_res$results_df[, c('first_type', 'second_type')]

if (!is.data.frame(rctd_14u_res_db)) {
    rctd_14u_res_db <- as.data.frame(as.matrix(rctd_14u_res_db))
}

if (!is.data.frame(celltype_firsec)) {
    celltype_firsec <- as.data.frame(celltype_firsec)
}

rctd_14u_res_db <- cbind(rctd_14u_res_db, celltype_firsec[rownames(rctd_14u_res_db), ])
colnames(rctd_14u_res_db) <- c('pct_firstType', 'pct_secondType', 'firstType', 'secondType')
head(rctd_14u_res_db)

rctdPct_fir <- rctd_14u_res_db[, c('pct_firstType', 'firstType')]
rctdPct_fir$bcd <- rownames(rctdPct_fir)  ## add bin barcode
rctdPct_sec <- rctd_14u_res_db[, c('pct_secondType', 'secondType')]
rctdPct_sec$bcd <- rownames(rctdPct_sec)  ## add bin barcode
colnames(rctdPct_fir) <- c('value', 'cellType', 'bcd_14u')
colnames(rctdPct_sec) <- c('value', 'cellType', 'bcd_14u')
rctdPct_mrg <- rbind( rctdPct_fir, rctdPct_sec )
head(rctdPct_mrg)

rctdPct_mrg_mtx <- reshape2::dcast(rctdPct_mrg, bcd_14u ~ cellType, fill = 0)
dim(rctdPct_mrg_mtx)
rctdPct_mrg_mtx[1:5, 1:5]

rownames(rctdPct_mrg_mtx) <- rctdPct_mrg_mtx[, 1]
rctdPct_mrg_mtx <- rctdPct_mrg_mtx[, -1]

write.csv(rctdPct_mrg_mtx,
          paste0('/csb2/finishedProject/HuiLiJian_DN_2025/10x_HD/5-final/2-RCTD/18s18941-T2.016um/RCTD_result.doublet.csv'))

,spot_class,first_type,second_type,first_class,second_class,min_score,singlet_score,conv_all,conv_doublet
,<fct>,<fct>,<fct>,<lgl>,<lgl>,<dbl>,<dbl>,<lgl>,<lgl>
s_016um_00290_00411-1,singlet,Tumor,Macrophage,FALSE,FALSE,257.2445,258.1739,TRUE,TRUE
s_016um_00329_00125-1,singlet,Tumor,Neutrophil,FALSE,FALSE,370.7768,372.8835,TRUE,TRUE
s_016um_00258_00092-1,singlet,Tumor,Macrophage,FALSE,FALSE,411.3215,419.2307,TRUE,TRUE
s_016um_00283_00339-1,singlet,Tumor,Smooth muscle,FALSE,FALSE,372.5167,374.2181,TRUE,TRUE
s_016um_00417_00360-1,singlet,Tumor,CAF,FALSE,FALSE,455.3016,463.5877,TRUE,TRUE
s_016um_00244_00289-1,singlet,Tumor,Smooth muscle,FALSE,FALSE,351.4504,355.7876,TRUE,TRUE


6 x 2 sparse Matrix of class "dgCMatrix"
                      first_type second_type
s_016um_00290_00411-1  0.9745245  0.02547549
s_016um_00329_00125-1  0.9585081  0.04149186
s_016um_00258_00092-1  0.9171169  0.08288307
s_016um_00283_00339-1  0.9713871  0.02861287
s_016um_00417_00360-1  0.9657755  0.03422447
s_016um_00244_00289-1  0.9359725  0.06402754

[1] 58965     2

,pct_firstType,pct_secondType,firstType,secondType
,<dbl>,<dbl>,<fct>,<fct>
s_016um_00290_00411-1,0.9745245,0.02547549,Tumor,Macrophage
s_016um_00329_00125-1,0.9585081,0.04149186,Tumor,Neutrophil
s_016um_00258_00092-1,0.9171169,0.08288307,Tumor,Macrophage
s_016um_00283_00339-1,0.9713871,0.02861287,Tumor,Smooth muscle
s_016um_00417_00360-1,0.9657755,0.03422447,Tumor,CAF
s_016um_00244_00289-1,0.9359725,0.06402754,Tumor,Smooth muscle


,value,cellType,bcd_14u
,<dbl>,<fct>,<chr>
s_016um_00290_00411-1,0.9745245,Tumor,s_016um_00290_00411-1
s_016um_00329_00125-1,0.9585081,Tumor,s_016um_00329_00125-1
s_016um_00258_00092-1,0.9171169,Tumor,s_016um_00258_00092-1
s_016um_00283_00339-1,0.9713871,Tumor,s_016um_00283_00339-1
s_016um_00417_00360-1,0.9657755,Tumor,s_016um_00417_00360-1
s_016um_00244_00289-1,0.9359725,Tumor,s_016um_00244_00289-1


[1] 58965    10

,bcd_14u,Neutrophil,CAF,Tumor,CD4T
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1,s_016um_00221_00036-1,0,0,0.6583038,0
2,s_016um_00221_00037-1,0,0,0.8194937,0
3,s_016um_00222_00013-1,0,0,0.9360318,0
4,s_016um_00222_00014-1,0,0,0.9105985,0
5,s_016um_00222_00015-1,0,0,0.9327489,0
